In [1]:
import os
import pandas as pd

In [2]:
herectvo = pd.read_json("herectvo_raw.json")

In [3]:
herectvo.shape

(86844, 12)

## Jen relevantní filmy

In [4]:
df = pd.read_json(os.path.join("data", "filmy.json"))

In [5]:
relevantni_filmy = df["Film"].to_list()

In [6]:
herectvo = herectvo[herectvo["Film"].isin(relevantni_filmy)]

In [7]:
herectvo.shape

(86648, 12)

## Doplnění dat z Wikidat

In [8]:
herectvo[herectvo["Datum narození"].isnull()].groupby("Jméno")["Film"].nunique().nlargest(20)

Jméno
Oldřich Vlach (14387)       85
Stanislav Litera (35280)    44
Igor Smržík (13197)         37
Jaroslav Vozáb (81363)      31
Jaroslav Šanda (11812)      31
Martin Sitta (847)          30
Josef Kubíček (27284)       28
František Stupka (11806)    26
Věra Křesadlová (22867)     22
Jiří Kalenský (7952)        21
Marcel Vašinka (9147)       20
Jitka Sedláčková (1435)     19
Setleři (129620)            18
Iva Hüttnerová (25411)      17
Jan Vostrčil (11100)        17
Růžena Kurelová (130454)    16
Jiří Maryško (81150)        15
Martin Kolár (7977)         15
Jan Diviš (126435)          14
Tomáš Turek (7085)          14
Name: Film, dtype: int64

In [9]:
bez_dat = {
"Oldřich Vlach (14387)": "Q12042457",
"Stanislav Litera (35280)": "Q101080646",
"Igor Smržík (13197)": "Q95172873",
"Jaroslav Vozáb (81363)": "Q12024124",
"Martin Sitta (847)": "Q61043065",
"Věra Křesadlová (22867)": "Q12001943",
"Jiří Kalenský (7952)": "Q12035254",
"Jitka Sedláčková (1435)": "Q12024960",
"Jan Vostrčil (11100)": "Q12023235",
"Jiří Maryško (81150)": "Q23765412",
"Iva Hüttnerová (25411)": "Q17227470",
"Martin Kolár (7977)": "Q65038076"
}

In [10]:
from wikidata.client import Client
import wikidata.entity

In [11]:
def wikidates(id):
    # zřejmě absurdně neefektivní kód, ale potřebuju ho jen jednou a z GPT-4 to fungovalo
    client = Client()  # doctest: +SKIP
    entity = client.get(id, load=True)
    P569_entity = wikidata.entity.Entity("P569", client)
    entity_dict = dict(entity)
    date_of_birth = entity_dict[P569_entity].strftime('%d. %m. %Y')
    try:
        P570_entity = wikidata.entity.Entity("P570", client)
        date_of_death = entity_dict[P570_entity].strftime('%d. %m. %Y')
    except KeyError:
        date_of_death = None
    
    return [date_of_birth, date_of_death]

In [12]:
for key, val in bez_dat.items():
    herectvo.loc[herectvo["Jméno"] == key,"Datum narození"]=wikidates(val)[0]
    herectvo.loc[herectvo["Jméno"] == key,"Datum úmrtí"]=wikidates(val)[1]
    print(f"{key}, {wikidates(val)}")

Oldřich Vlach (14387), ['03. 07. 1941', None]
Stanislav Litera (35280), ['11. 06. 1931', '21. 09. 2003']
Igor Smržík (13197), ['09. 06. 1949', None]
Jaroslav Vozáb (81363), ['16. 12. 1919', '03. 04. 1988']
Martin Sitta (847), ['18. 11. 1968', None]
Věra Křesadlová (22867), ['28. 02. 1944', None]
Jiří Kalenský (7952), ['02. 10. 1954', None]
Jitka Sedláčková (1435), ['18. 03. 1961', None]
Jan Vostrčil (11100), ['03. 12. 1903', '25. 01. 1985']
Jiří Maryško (81150), ['04. 04. 1980', None]
Iva Hüttnerová (25411), ['17. 12. 1948', None]
Martin Kolár (7977), ['27. 07. 1958', '24. 08. 2010']


## Data a místa

In [13]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

In [14]:
herectvo["Město narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
herectvo["Stát narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Město úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Stát úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])

In [15]:
herectvo.loc[herectvo["Jméno"] == "Bolek Polívka (1556)", "Jméno"] = "Boleslav Polívka (1556)"

In [16]:
herectvo.loc[herectvo["Jméno"] == "Eva Pecková (34563)", "Datum narození"] = None

In [17]:
herectvo["Rok narození"] = herectvo["Datum narození"].astype(str).apply(lambda x: x[-4:])
herectvo["Rok narození"] = pd.to_numeric(herectvo["Rok narození"], errors='coerce') 

In [18]:
herectvo["Datum narození"] = pd.to_datetime(herectvo["Datum narození"], errors="ignore", format = "%d.%m.%Y")

In [19]:
herectvo["Rok úmrtí"] = herectvo["Datum úmrtí"].astype(str).apply(lambda x: x[-4:])
herectvo["Rok úmrtí"] = pd.to_numeric(herectvo["Rok úmrtí"], errors='coerce') 

In [20]:
herectvo["Datum úmrtí"] = pd.to_datetime(herectvo["Datum úmrtí"], errors="ignore", format = "%d.%m.%Y")

In [21]:
herectvo["Věk"] = herectvo["Rok"].astype(float) - herectvo["Rok narození"].astype(float)

## Uložení

In [23]:
herectvo.to_json(os.path.join("data", "herectvo.json"), orient="records")

In [24]:
import zipfile
try:
    import zlib
    mode= zipfile.ZIP_DEFLATED
except:
    mode= zipfile.ZIP_STORED

In [25]:
with zipfile.ZipFile(os.path.join("data", "herectvo.zip"), mode="w") as my_zip:
    my_zip.write(os.path.join("data", "herectvo.json"), compress_type=zipfile.ZIP_DEFLATED)